In [1]:
import os
import numpy as np
from skimage import io, measure
from skimage.measure import regionprops
import glob
from ultralytics import YOLO

import torch
import napari
import skimage as ski
import pandas as pd
import plotly.express as px

In [2]:
viewer = napari.Viewer()

# Test Model

In [3]:
#fnames = glob.glob('data/val/images/*.tif')
# 01_38_02 and 02_04_03 are the interesting ones
fnames = glob.glob('new_data/test/images/*.tif')
#fnames = glob.glob('S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719/*.tif')
fnames

['S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_01_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_01_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_02_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_02_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_03_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_03_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_04_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_04_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_05_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_05_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_06_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_06_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_07_02.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\\01_07_03.tif',
 'S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130

In [4]:
fname = fnames[1]
print(fname)

model = YOLO("runs/detect/train36/weights/best.pt")  # load a custom model

results = model(ski.io.imread(fname))
results[0].boxes[0]

height, width = ski.io.imread(fname).shape[:2]
confs = []

rtn_boxes = []
classes = []
for box in results[0].boxes:
    xyxy = np.reshape(box.xyxy[0].cpu().numpy(), [-1,2])[:,::-1]
    rtn_boxes.append(xyxy)
    classes.append(box.cls.cpu().numpy()[0])
    confs.append(box.conf[0].cpu().numpy())

img = ski.io.imread(fname)

classes = np.array(classes)
rtn_boxes = np.array(rtn_boxes)
confs = np.array(confs)

rtn_boxes = rtn_boxes[confs > 0.05]
classes = classes[confs > 0.05]

viewer = napari.Viewer()

viewer.add_image(img, channel_axis=-1, colormap=['red', 'green', 'blue'])

viewer.add_shapes(rtn_boxes[classes==0], shape_type='rectangle', edge_color='red', face_color='red', opacity=0.5, name='T0')
viewer.add_shapes(rtn_boxes[classes==1], shape_type='rectangle', edge_color='blue', face_color='blue', opacity=0.5, name='T1')
viewer.add_shapes(rtn_boxes[classes==2], shape_type='rectangle', edge_color='green', face_color='green', opacity=0.5, name='T2')
viewer.add_shapes(rtn_boxes[classes==3], shape_type='rectangle', edge_color='yellow', face_color='yellow', opacity=0.5, name='T3')

S:/micro/nro/fx2482/lem/20240820_OSS_IMARE-130719\01_01_03.tif

0: 1600x1600 70 1_stage_IBs, 19 2_stage_IIs, 47 4_stage_late_IIIs, 33.3ms
Speed: 301.5ms preprocess, 33.3ms inference, 0.0ms postprocess per image at shape (1, 3, 1600, 1600)


<Shapes layer 'T3' at 0x19936c3f310>

<Shapes layer 'T3' at 0x25d405f4d00>

# Manually fix and export

In [24]:
layer_names = [layer.name for layer in viewer.layers[3:]]
for layer in layer_names:
    rects = viewer.layers[layer].data
    np.save(fname.replace('.tif', f'_{layer}.npy'), rects)